<h1>KBG Syndrome</h1>
<p>Data from <a href="https://pubmed.ncbi.nlm.nih.gov/36446582/" target="__blank">Martinez-Cayuelas E, et al. Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients. J Med Genet. 2022 Nov 29:jmedgenet-2022-108632. PMID: 36446582.</a>.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import numpy as np
from google.protobuf.json_format import MessageToDict, MessageToJson
import os
import sys
import re
from pyliftover import LiftOver

sys.path.append('../')
from pyphetools.creation import *

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0003-2598-6622")
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
data = pd.read_excel('input/Martinez-KBG-SupplTable-340-v2.xlsx', dtype=np.str_)

In [4]:
data.head()

,Patient origin (1=our cohort; 2=literature),Patient ID,Gender (1=male; 2=female),ID,ASD,ADHD,History of developmental delay (motor and language >18m),ID/ADHD/ASD,Macrodontia and/or other dental anomalies,"Characteristic nose (anteverted, bulbous, and/or prominent)",...,Postnatal short stature <p10,N comorbidities (N=7),Hearing Loss and/or otitis media,"Other comorbidities (seizures, cardiopathy, visual, feeding, cryptorchydism)",Phenotypic score (columns F:P),Variant type (SNV vs CNV),Exon 9 (Yes/No),c.1903_1907del;p.Lys635GInfs*26 (Yes/No),Deletion size (CNV),Variant
0,1,KBG1,1,Yes,NaN,NaN,Yes,Yes,NaN,NaN,...,NaN,3,NaN,Yes,8,CNV,CNV,CNV,17000,16q24.3(89336307_89354085)x1
1,1,KBG2,1,Yes,NaN,NaN,No,Yes,NaN,NaN,...,NaN,1,NaN,Yes,4,CNV,CNV,CNV,250000,16q24.3(89256478_89506223)x1
2,1,KBG3,2,NaN,NaN,NaN,Yes,NaN,NaN,NaN,...,NaN,3,Yes,Yes,5,SNV,Yes,No,SNV,c.2398_2401del;p.Glu800Asnfs*62
3,1,KBG4,2,Yes,NaN,NaN,Yes,Yes,NaN,No,...,No,1,No,Yes,6,SNV,Yes,No,SNV,c.7083del;p.Thr2362Profs*39
4,1,KBG5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,SNV,Yes,Yes,SNV,c.1903_1907del;p.Lys635GInfs*26


In [5]:
items = {
    'ID':['Intellectual disability','HP:0001249'],
    'ASD':['Autistic behavior','HP:0000729'],
    'ADHD':['Attention deficit hyperactivity disorder', 'HP:0007018'],
    'History of developmental delay (motor and language >18m)': ['Neurodevelopmental delay', 'HP:0012758'],
    'ID/ADHD/ASD':['Abnormality of higher mental function', 'HP:0011446'],
    'Macrodontia and/or other dental anomalies':['Abnormality of dental morphology', 'HP:0006482'],
    'Characteristic nose (anteverted, bulbous, and/or prominent)': ['Abnormal external nose morphology', 'HP:0010938'],
    'Triangular face': ['Triangular face', 'HP:0000325'],
    'Characteristic eyebrows': ['Abnormal eyebrow morphology', 'HP:0000534'],
    'Long philtrum': ['Long philtrum', 'HP:0000343'],
    'Characteristic ears (large, prominent, and/or low-set)': ['Abnormality of the outer ear', 'HP:0000356'],
    'Hand anomalies (brachydactyly or clinodactyly)': ['Abnormality of the hand', 'HP:0001155'],
    'Postnatal short stature <p10': ['Short stature', 'HP:0004322'],
    'Hearing Loss and/or otitis media': ['Hearing impairment', 'HP:0000365']
}

In [6]:
column_mapper_d = defaultdict(ColumnMapper)

item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, observed='Yes', excluded='No', non_measured='NA')
print(f"We created {len(item_column_mapper_d)} simple column mappers")

for k, v in item_column_mapper_d.items():
    column_mapper_d[k] = v



We created 14 simple column mappers


In [7]:
lift = LiftOver('hg19', 'hg38')

In [9]:
count = 0

#URL = "https://rest.variantvalidator.org/VariantValidator/variantvalidator/%s/%s%%3A%s/%s?content-type=application%%2Fjson"

tempArray = []
for val in data['Variant']: 
    want = re.findall(r'(.*);', str(val))
    if len(want) == 0:
        print(val)
        want = re.findall(r'.*\((\d*)[-|_](\d*)\)x.*', str(val))
        print(want)
        if len(want) == 0:
            finalWant = str(val)
        else:
            want = want[0]
            coord1 = want[0]
            coord2 = want[1]
            liftOut1 = lift.convert_coordinate(chromosome='chr16', position=int(coord1))
            liftOut2 = lift.convert_coordinate(chromosome='chr16', position=int(coord2))
            newCoord1 = str(liftOut1[0][1])
            newCoord2 = str(liftOut2[0][1])
            finalWant1 = re.sub(coord1, newCoord1, str(val))
            finalWant = re.sub(coord2, newCoord2, finalWant1)
            print(finalWant)
    else:
        finalWant = want[0]
    tempArray.append(finalWant)
    count = count +1

data['BetterVariants'] = tempArray
print(data['BetterVariants'])


16q24.3(89336307_89354085)x1
[('89336307', '89354085')]
16q24.3(89269899_89287677)x1
16q24.3(89256478_89506223)x1
[('89256478', '89506223')]
16q24.3(89190070_89439815)x1
16q24.3(88854757_89520963)x1
[('88854757', '89520963')]
16q24.3(88788349_89454555)x1
16q24.3(89349711_89607741)x1
[('89349711', '89607741')]
16q24.3(89283303_89541333)x1
c.227G>A
[]
16q24.3(89283689-89579130)x1
[('89283689', '89579130')]
16q24.3(89217281-89512722)x1
16q24.3(89359701_89379306)x1
[('89359701', '89379306')]
16q24.3(89293293_89312898)x1
16q24.3(89325387_89409147)x1
[('89325387', '89409147')]
16q24.3(89258979_89342739)x1
16q24.3(89556618_89557004)x1
[('89556618', '89557004')]
16q24.3(89490210_89490596)x1
16q24.3(89261399_89419131)x1
[('89261399', '89419131')]
16q24.3(89194991_89352723)x1
16q24.3(88854757_89577705)x1
[('88854757', '89577705')]
16q24.3(88788349_89511297)x1
16q24.3(89542695_89656251)x3
[('89542695', '89656251')]
16q24.3(89476287_89589843)x3
16q24.3(89350931_89439639)x3
[('89350931', '89439639'

In [10]:
data['BetterVariants']

0           16q24.3(89269899_89287677)x1
1           16q24.3(89190070_89439815)x1
2                         c.2398_2401del
3                              c.7083del
4                         c.1903_1907del
                     ...                
335         16q24.3(89056331_89434622)x1
336         16q24.3(89206684_89472351)x1
337    16q24.2-24.3(87464658_89530534)x1
338    16q24.2-24.3(88197355_89297194)x1
339         16q24.3(87886394-88066394)x1
Name: BetterVariants, Length: 340, dtype: object

In [11]:
genome = 'hg38'
default_genotype = 'heterozygous'
transcript = 'NM_013275.6'
varMapper = VariantColumnMapper(assembly=genome,column_name='BetterVariants', 
                                transcript=transcript, default_genotype=default_genotype, is_structural_symbol='CNV',
                               structure_column='Variant type (SNV vs CNV)')

In [15]:
sexMapper = SexColumnMapper(male_symbol='1', female_symbol='2', unknown_symbol='NA', column_name='Gender (1=male; 2=female)')
ageMapper = AgeColumnMapper.by_year('Patient origin (1=our cohort; 2=literature)')
sexMapper.preview_column(data['Gender (1=male; 2=female)'])

Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan


,original column contents,sex
0,1,MALE
1,1,MALE
2,2,FEMALE
3,2,FEMALE
4,1,MALE
...,...,...
335,1,MALE
336,1,MALE
337,1,MALE
338,1,MALE


In [21]:
individual_colname = 'Patient ID'
pmid = 'PMID:36446582'

In [22]:
encoder = CohortEncoder(df=data, hpo_cr=hpo_cr, column_mapper_d=column_mapper_d, 
                          individual_column_name=individual_colname,
                            sexmapper=sexMapper,
                        agemapper=ageMapper,
                            variant_mapper=varMapper,
                        metadata=metadata,
                       pmid=pmid)

In [23]:
encoder.set_disease(disease_id='OMIM:148050', label='KBG syndrome')

In [24]:
encoder.preview_dataframe()

Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan
Could not map sex symbol nan


,sex,age,phenotypic features
id,,,
KBG1,MALE,P1Y,Intellectual disability (HP:0001249)\nnot measured: Autistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nNeurodevelopmental delay (HP:0012758)\nAbnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nnot measured: Abnormal external nose morphology (HP:0010938)\nTriangular face (HP:0000325)\nAbnormal eyebrow morphology (HP:0000534)\nLong philtrum (HP:0000343)\nexcluded: Abnormality of the outer ear (HP:0000356)\nnot measured: Abnormality of the hand (HP:0001155)\nnot measured: Short stature (HP:0004322)\nnot measured: Hearing impairment (HP:0000365)
KBG2,MALE,P1Y,Intellectual disability (HP:0001249)\nnot measured: Autistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nexcluded: Neurodevelopmental delay (HP:0012758)\nAbnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nnot measured: Abnormal external nose morphology (HP:0010938)\nTriangular face (HP:0000325)\nAbnormal eyebrow morphology (HP:0000534)\nexcluded: Long philtrum (HP:0000343)\nnot measured: Abnormality of the outer ear (HP:0000356)\nnot measured: Abnormality of the hand (HP:0001155)\nnot measured: Short stature (HP:0004322)\nnot measured: Hearing impairment (HP:0000365)
KBG3,FEMALE,P1Y,not measured: Intellectual disability (HP:0001249)\nnot measured: Autistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nNeurodevelopmental delay (HP:0012758)\nnot measured: Abnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nnot measured: Abnormal external nose morphology (HP:0010938)\nnot measured: Triangular face (HP:0000325)\nAbnormal eyebrow morphology (HP:0000534)\nexcluded: Long philtrum (HP:0000343)\nnot measured: Abnormality of the outer ear (HP:0000356)\nnot measured: Abnormality of the hand (HP:0001155)\nnot measured: Short stature (HP:0004322)\nHearing impairment (HP:0000365)
KBG4,FEMALE,P1Y,Intellectual disability (HP:0001249)\nnot measured: Autistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nNeurodevelopmental delay (HP:0012758)\nAbnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nexcluded: Abnormal external nose morphology (HP:0010938)\nTriangular face (HP:0000325)\nAbnormal eyebrow morphology (HP:0000534)\nLong philtrum (HP:0000343)\nnot measured: Abnormality of the outer ear (HP:0000356)\nexcluded: Abnormality of the hand (HP:0001155)\nexcluded: Short stature (HP:0004322)\nexcluded: Hearing impairment (HP:0000365)
KBG5,MALE,P1Y,not measured: Intellectual disability (HP:0001249)\nnot measured: Autistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nnot measured: Neurodevelopmental delay (HP:0012758)\nnot measured: Abnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nnot measured: Abnormal external nose morphology (HP:0010938)\nnot measured: Triangular face (HP:0000325)\nnot measured: Abnormal eyebrow morphology (HP:0000534)\nexcluded: Long philtrum (HP:0000343)\nnot measured: Abnormality of the outer ear (HP:0000356)\nnot measured: Abnormality of the hand (HP:0001155)\nnot measured: Short stature (HP:0004322)\nnot measured: Hearing impairment (HP:0000365)
...,...,...,...
Willemsen2010_P1,MALE,P2Y,Intellectual disability (HP:0001249)\nAutistic behavior (HP:0000729)\nnot measured: Attention deficit hyperactivity disorder (HP:0007018)\nnot measured: Neurodevelopmental delay (HP:0012758)\nAbnormality of higher mental function (HP:0011446)\nnot measured: Abnormality of dental morphology (HP:0006482)\nnot measured: Abnormal external nose morphology (HP:0010938)\nexcluded: Triangular face (HP:0000325)\nnot 

In [26]:
encoder.output_phenopackets(outdir='phenopackets-v2')

https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.2398_2401del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7083del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1903_1907del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7407C>G/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6691dup/NM_013275.6?content-type=application%2Fjson
https://rest.varian

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6628G>T/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7753C>T/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6792dup/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1903_1907del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.or

https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.3774_3775del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.2395A>T/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1389dup/NM_013275.6?content-type=application%2Fjson
Could not map sex symbol nan
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7552C>T/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.2828_2829del/NM_013275.6?content-type=application%2Fjson
Could not map sex symbol nan
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.6340C>T/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.3045del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.4406G>A/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1903_1907del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1173C>G/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/Varian

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.7411_7422del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.1903_1907del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.211_226+1del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.5238_5239del/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/tools/gene2transcripts_v2/NM_013275.6/NM_013275.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_013275.6%3Ac.5488G>T/NM_013275.6?content-type=application%2Fjson
https://r